In [72]:
import mlflow
import pandas as pd

from cookiemonster.utils import LOGS_PATH

In [73]:
from mlflow.tracking import MlflowClient

tracking_uri = str(LOGS_PATH.joinpath("mlflow"))

mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient(
    tracking_uri=tracking_uri
)

In [74]:
experiment_name="bias_detection_09-04_15-00"
runs = mlflow.search_runs(experiment_names=[experiment_name])
run_ids = list(runs.run_id)
run_ids

['dfbdd28ca90c47bb93f7f4d981c04f69',
 '96828147a47b493987fb90ff5c24574f',
 'a6aeac653b1648d584bf8112da0f5502',
 'a63c5caf467c47e8a98506dcc0ec8e16',
 '6c6345e71d9d43c7b7de3dfae4e24234',
 '217359fcd7504ca0bef4897ed972a842']

In [75]:
data = []
metric_names = ["rmsre"]
param_names = ["user.baseline", "user.bias_detection_knob"]

for run_id in run_ids:
    
    run_params = client.get_run(run_id).data.params
    
    metric_histories = {}
    for metric_name in metric_names:
        metric_histories[metric_name] = client.get_metric_history(run_id, metric_name)
    
    for step in range(len(metric_histories[metric_names[0]])):
        step_data = {
            "step": step,
        }
        for metric_name in metric_names:
            step_data[metric_name] = metric_histories[metric_name][step].value        
    
        for param_name in param_names:
            step_data[param_name] = run_params[param_name]
    
        data.append(step_data)

df = pd.DataFrame(data)
df

,step,rmsre,user.baseline,user.bias_detection_knob
0,0,0.015920,cookiemonster_base,0
1,1,0.015903,cookiemonster_base,0
2,2,0.015838,cookiemonster_base,0
3,3,0.015887,cookiemonster_base,0
4,4,0.015895,cookiemonster_base,0
...,...,...,...,...
3001,496,0.158955,cookiemonster,0.5
3002,497,0.161519,cookiemonster,0.5
3003,498,0.158830,cookiemonster,0.5
3004,499,0.174944,cookiemonster,0.5


In [76]:
# df["baseline"] = df["user.baseline"]
# df["bias_detection_knob"] = df["user.bias_detection_knob"]

def get_baseline_name(row):
    if row["user.bias_detection_knob"] == "0":
        return row["user.baseline"]
    return row["user.baseline"] + "_bias" + row["user.bias_detection_knob"]

df["baseline"] = df.apply(get_baseline_name, axis=1)



In [77]:
df.baseline.unique()

array(['cookiemonster_base', 'cookiemonster', 'ipa',
       'cookiemonster_bias0.1', 'cookiemonster_bias1',
       'cookiemonster_bias0.5'], dtype=object)

In [78]:
from plotting.macros import *
from plotting.plot_template import *


In [79]:
cdf_args = {
        "df": df,
        "metric": "rmsre",
        "x_axis": None,
        "x_axis_title": RMSRE_CDF_X,
        "y_axis_title": RMSRE_Y,
        "ordering": None,
        "log_y": False,
        "x_range": [1, 100],
        "showlegend": False,
        "marker_pos": 0.98,
    }

In [80]:
figs_args = {
    "axis_title_font_size": {"x": 18, "y": 18},
    "axis_tick_font_size": {"x": 14, "y": 14},
    "legend": {
        "yanchor": "top",
        "y": 1.2,
        "xanchor": "left",
        "x": 0.2,
        "orientation": "h",
    },
    "output_path": f"cdf_rmsre_{experiment_name}.png",
    "height": 300,
    "width": 1500,
}
make_plots([(cdf, cdf_args)], cols=1, **figs_args)


In [81]:
# TODO: plot this.